In [1]:
import requests
import pandas as pd
import glob

In [2]:
# récupérer tous les fichiers du dossier videos et dédupliquer les lignes
files = glob.glob("market_cinema_fr.csv")

df = []
for f in files:
    csv = pd.read_csv(f)
    df.append(csv)
df = pd.concat(df,ignore_index=True)
df.drop_duplicates(subset=['video_id'], keep="first", inplace=True)
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   video_id      1000 non-null   str  
 1   title         1000 non-null   str  
 2   description   819 non-null    str  
 3   channel       1000 non-null   str  
 4   published_at  1000 non-null   str  
 5   duration      999 non-null    str  
 6   views         1000 non-null   int64
 7   likes         1000 non-null   int64
 8   comments      1000 non-null   int64
 9   channel_id    1000 non-null   str  
 10  category_id   1000 non-null   int64
 11  language      1000 non-null   str  
dtypes: int64(4), str(8)
memory usage: 93.9 KB


In [3]:
# langues: supprimer les lignes avec valeurs
df.dropna(subset="language",axis=0,inplace=True)

# langues: regrouper fr_fr, be_fr sous 'fr', pareil pour 'en'
df.loc[df['language'].str.contains('^fr'),'language'] = "fr"
df.loc[df['language'].str.contains('^en'),'language'] = "en"

In [4]:
#supprimer autres langues
df['language'].value_counts()

language
fr       868
en        79
de        32
ar         6
zxx        3
es-ES      2
nl         2
zh         2
hi         1
und        1
lb         1
th         1
no         1
wo         1
Name: count, dtype: int64

In [5]:
# calculer le taux d'engagement pour voir en un coup d'oeil les vidéos ayant reçu le plus d'intéractions
df["Engagement total"] = df["likes"] + df["comments"]
df["Taux d'engagement (%)"] = (df["Engagement total"] / df["views"].replace(0, 1)) * 100

In [6]:
# transofrmer les durées (ici min/sec) en secondes (plus simple pour l'analyse
import re

# 1. Conversion en timedelta (errors='coerce' transforme les formats invalides en NaT)
df['duration_td'] = pd.to_timedelta(df['duration'], errors='coerce')

# 2. Conversion en secondes
# .astype('Int64') permet de garder les NaN si nécessaire sans faire planter le code
df['Durée (s)'] = df['duration_td'].dt.total_seconds().astype('Int64')

# OPTIONNEL : Si vous voulez remplacer les vides par 0 et avoir des entiers classiques
# df['Durée (s)'] = df['duration_td'].dt.total_seconds().fillna(0).astype(int)

print(df[['duration', 'Durée (s)']])

     duration  Durée (s)
0        PT1M         60
1     PT9M11S        551
2      PT1M4S         64
3     PT1M32S         92
4      PT6M2S        362
..        ...        ...
995     PT42S         42
996   PT1M32S         92
997  PT12M21S        741
998      PT1M         60
999  PT12M24S        744

[1000 rows x 2 columns]


In [7]:
# colonnes avec valeurs nulls
df.isna().sum()

video_id                   0
title                      0
description              181
channel                    0
published_at               0
duration                   1
views                      0
likes                      0
comments                   0
channel_id                 0
category_id                0
language                   0
Engagement total           0
Taux d'engagement (%)      0
duration_td                1
Durée (s)                  1
dtype: int64

In [8]:
df.to_csv("cinema_clean.csv", index=False)